## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kabinda,CD,-6.1333,24.4833,67.89,84,43,3.29,scattered clouds
1,1,Sassandra,CI,4.9500,-6.0833,79.20,89,42,4.83,scattered clouds
2,2,Navalmoral De La Mata,ES,39.8916,-5.5406,56.26,54,0,1.41,clear sky
3,3,Kachiry,KZ,53.0675,76.1036,34.23,97,100,14.61,overcast clouds
4,4,Kruisfontein,ZA,-34.0033,24.7314,60.01,94,100,4.00,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
condition = 0
while condition ==0:
    try:
        min_temp = float(input("What is the minimum temperature you would like for your trip? "))
        max_temp = float(input("What is the maximum temperature you would like for your trip? "))
        if max_temp < min_temp:
            max_temp, min_temp = min_temp, max_temp
        condition = 1
    except:
        print("Pleae input number")

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[city_data_df['Max Temp'].between(min_temp,max_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sassandra,CI,4.9500,-6.0833,79.20,89,42,4.83,scattered clouds
5,5,Nouakchott,MR,18.0858,-15.9785,75.20,57,7,18.41,clear sky
8,8,Vao,NC,-22.6667,167.4833,76.21,66,94,20.20,overcast clouds
9,9,Ipubi,BR,-7.6519,-40.1489,79.21,52,82,3.20,broken clouds
12,12,Muzaffargarh,PK,30.0754,71.1921,75.20,53,75,6.91,smoke


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 1 to 712
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           239 non-null    int64  
 1   City                 239 non-null    object 
 2   Country              238 non-null    object 
 3   Lat                  239 non-null    float64
 4   Lng                  239 non-null    float64
 5   Max Temp             239 non-null    float64
 6   Humidity             239 non-null    int64  
 7   Cloudiness           239 non-null    int64  
 8   Wind Speed           239 non-null    float64
 9   Current Description  239 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 20.5+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sassandra,CI,79.20,scattered clouds,4.9500,-6.0833,
5,Nouakchott,MR,75.20,clear sky,18.0858,-15.9785,
8,Vao,NC,76.21,overcast clouds,-22.6667,167.4833,
9,Ipubi,BR,79.21,broken clouds,-7.6519,-40.1489,
12,Muzaffargarh,PK,75.20,smoke,30.0754,71.1921,
14,Lethem,GY,84.70,overcast clouds,3.3803,-59.7968,
15,Bonthe,SL,80.44,clear sky,7.5264,-12.5050,
16,Jaciara,BR,84.11,moderate rain,-15.9653,-54.9683,
17,Praya,CV,75.20,clear sky,14.9215,-23.5087,
20,Nsawam,GH,84.20,few clouds,5.8089,-0.3503,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for idx,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat, lng = row["Lat"], row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    baseurl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    resp = requests.get(baseurl, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[idx, "Hotel Name"] = resp["results"][0]["name"]
    except:
        print(f"Hotel not found for city {row['City']}...skipping.")    
        

Hotel not found for city Adre...skipping.
Hotel not found for city Gayeri...skipping.
Hotel not found for city Poya...skipping.
Hotel not found for city Taoudenni...skipping.
Hotel not found for city Birao...skipping.
Hotel not found for city Gao...skipping.
Hotel not found for city Lagos...skipping.
Hotel not found for city Maraa...skipping.
Hotel not found for city Nokaneng...skipping.
Hotel not found for city Marovoay...skipping.
Hotel not found for city Amapa...skipping.
Hotel not found for city Bauchi...skipping.
Hotel not found for city Barentu...skipping.
Hotel not found for city Bonfim...skipping.
Hotel not found for city Vadlapudi...skipping.
Hotel not found for city Adrar...skipping.
Hotel not found for city Araouane...skipping.
Hotel not found for city Wagar...skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"]==""].index)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sassandra,CI,79.20,scattered clouds,4.9500,-6.0833,la terrasse
5,Nouakchott,MR,75.20,clear sky,18.0858,-15.9785,HOTEL IMAN فندق إيمان
8,Vao,NC,76.21,overcast clouds,-22.6667,167.4833,Hotel Kou-Bugny
9,Ipubi,BR,79.21,broken clouds,-7.6519,-40.1489,Só Love Motel
12,Muzaffargarh,PK,75.20,smoke,30.0754,71.1921,Aghosh Hotel


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index=False, index_label="City")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))